### **Data Modeling Proccess**:
- Objective: This notebook presents the process of creating a dimensional model based on optimization, scalability, practicality, performance and, of course, data and its nature. 
---

#### **First Step**: Get data directly from the database

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Fetch the data from the database as a dataframe
df = pd.read_csv('data.csv')

In [ ]:
# Check the shape
df.shape

(1052352, 26)

In [34]:
#sampling data
df_sample = df.sample(n=300000, random_state=42)
df_sample.to_csv(os.path.join(os.getcwd(), '..', 'data', 'sample_credit_card_transactions_api_preprocessed.csv'), index=False, encoding='utf-8',sep=",")

*At this point we perform a sample, however in Airflow it will be executed with the complete dataset.*

---

# Working with Sample Data

In [36]:
# Remove the column display limit to show all columns in the DataFrame
pd.set_option('display.max_columns', None)

In [37]:
#path to sample data
path_to_data_sample = os.path.join(os.getcwd(), '..', 'data', 'credit_card_transactions_api_preprocessed.csv')

#loading the sample data and checking the first 7 rows
df_sample = pd.read_csv(path_to_data_sample)
df_sample.head()

,id,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,job,dob,trans_num,is_fraud,merch_zipcode,age,country,country_code,state_abbreviation,state_name,state_population
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.0788,-81.1781,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,False,28705.0,31,United States,USA,NC,North Carolina,10439459
1,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,ID,83252,42.1808,-112.2620,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,False,83236.0,57,United States,USA,ID,Idaho,1839117
2,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,VA,24433,38.4207,-79.4629,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,False,22844.0,33,United States,USA,VA,Virginia,8631373
3,5,2019-01-01 00:04:08,4767265376804500,"fraud_Stroman, Hudson and Erdman",gas_transport,94.63,Jennifer,Conner,F,4655 David Island,Dublin,PA,18917,40.3750,-75.2045,Transport planner,1961-06-19,189a841a0a8ba03058526bcfe566aab5,False,17972.0,58,United States,USA,PA,Pennsylvania,13002788
4,7,2019-01-01 00:05:08,6011360759745864,fraud_Corwin-Collins,gas_transport,71.65,Steven,Williams,M,231 Flores Pass Suite 720,Edinburg,VA,22824,38.8432,-78.6003,"Designer, multimedia",1947-08-21,6d294ed2cc447d2c71c7171a3d54967c,False,22644.0,72,United States,USA,VA,Virginia,8631373


## Previous Transformations to split the data

In [38]:
"""
Changing the transaction date to an incremental integer
"""
# making sure the data type is datetime
df_sample['trans_date_trans_time'] = pd.to_datetime(df_sample['trans_date_trans_time'])

# converting the transaction date to a incremental integer
df_sample['trans_date_id'] = df_sample['trans_date_trans_time'].dt.strftime('%Y%m%d').astype(int)

In [39]:
"""
Retrieve FIPS code for states using 'us' library
"""
import us

def get_fips_from_usps(usps_code):
    """
    This function takes a two-letter USPS code (state abbreviation) and returns the corresponding FIPS code.
    Args:
        usps_code (str): The two-letter state abbreviation (e.g., 'CA', 'NY').
    Returns:
        str: The FIPS code corresponding to the state, or None if the USPS code is not found.
    """
    state = us.states.lookup(usps_code)
    if state:
        return state.fips
    else:
        return None  # Returns None if the USPS code is not found

# Apply the function to generate a new column 'state_id' with the FIPS codes
df_sample['state_id'] = df_sample['state_abbreviation'].apply(get_fips_from_usps)



In [40]:
"""
Creating a unique ID for each category
"""

# Create a dictionary that maps each unique 'category' value to an incremental number
category_mapping = {category: idx for idx, category in enumerate(df_sample['category'].unique(), start=0)}

# Use the dictionary to create the 'category_id' column
df_sample['category_id'] = df_sample['category'].map(category_mapping)


In [41]:
"""
Creating a unique ID for each job
"""

# Create a dictionary that maps each unique 'category' value to an incremental number
job_mapping = {job: idx for idx, job in enumerate(df_sample['job'].unique(), start=1010)}

# Use the dictionary to create the 'job_id' column
df_sample['job_id'] = df_sample['job'].map(job_mapping)


In [42]:
"""
Location ID generation by concatenating 'zip', 'state_id', and an incrementing number
"""
counter = 1
def generate_unique_id(row):
    global counter
    unique_id = str(row['state_id']) + str(row['zip']) + str(counter)
    counter += 1  # Increment the counter for the next row
    return unique_id

# Apply the function to generate the 'unique_id' column in df_sample
df_sample['location_id'] = df_sample.apply(generate_unique_id, axis=1)

In [43]:
# Count the number of duplicates in the 'unique_id' column
num_duplicates = df_sample['location_id'].duplicated().sum()
num_duplicates

np.int64(0)

In [44]:
# Function to split the 'trans_date_trans_time' column into multiple new columns
def split_datetime_column(df):
    # Create 'date' column (YYYY-MM-DD format)
    df['date'] = pd.to_datetime(df['trans_date_trans_time']).dt.date
    
    # Create 'year' column (YYYY)
    df['year'] = pd.to_datetime(df['trans_date_trans_time']).dt.year
    
    # Create 'month' column (MM)
    df['month'] = pd.to_datetime(df['trans_date_trans_time']).dt.month
    
    # Create 'quarter' column (Q)
    df['quarter'] = pd.to_datetime(df['trans_date_trans_time']).dt.quarter
    
    # Create 'hour' column (HH:MM:SS)
    df['hour'] = pd.to_datetime(df['trans_date_trans_time']).dt.time
    
    return df

df_sample = split_datetime_column(df_sample)

In [45]:
#convert 'merch_zipcode' to integer
df_sample['merch_zipcode'] = df_sample['merch_zipcode'].apply(lambda x: int(x))

In [46]:
# verifying the changes
df_sample.head()

,id,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,job,dob,trans_num,is_fraud,merch_zipcode,age,country,country_code,state_abbreviation,state_name,state_population,trans_date_id,state_id,category_id,job_id,location_id,date,year,month,quarter,hour
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.0788,-81.1781,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,False,28705,31,United States,USA,NC,North Carolina,10439459,20190101,37,0,1010,37286541,2019-01-01,2019,1,1,00:00:18
1,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,ID,83252,42.1808,-112.2620,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,False,83236,57,United States,USA,ID,Idaho,1839117,20190101,16,1,1011,16832522,2019-01-01,2019,1,1,00:00:51
2,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,VA,24433,38.4207,-79.4629,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,False,22844,33,United States,USA,VA,Virginia,8631373,20190101,51,2,1012,51244333,2019-01-01,2019,1,1,00:03:06
3,5,2019-01-01 00:04:08,4767265376804500,"fraud_Stroman, Hudson and Erdman",gas_transport,94.63,Jennifer,Conner,F,4655 David Island,Dublin,PA,18917,40.3750,-75.2045,Transport planner,1961-06-19,189a841a0a8ba03058526bcfe566aab5,False,17972,58,United States,USA,PA,Pennsylvania,13002788,20190101,42,3,1013,42189174,2019-01-01,2019,1,1,00:04:08
4,7,2019-01-01 00:05:08,6011360759745864,fraud_Corwin-Collins,gas_transport,71.65,Steven,Williams,M,231 Flores Pass Suite 720,Edinburg,VA,22824,38.8432,-78.6003,"Designer, multimedia",1947-08-21,6d294ed2cc447d2c71c7171a3d54967c,False,22644,72,United States,USA,VA,Virginia,8631373,20190101,51,3,1014,51228245,2019-01-01,2019,1,1,00:05:08


# Dimension Creation

In [47]:
fact_T_transation_dim = df_sample[['trans_num','is_fraud','amt','hour','trans_date_id','trans_date_trans_time','cc_num','location_id','merch_zipcode']]
fact_T_transation_dim.head()

,trans_num,is_fraud,amt,hour,trans_date_id,trans_date_trans_time,cc_num,location_id,merch_zipcode
0,0b242abb623afc578575680df30655b9,False,4.97,00:00:18,20190101,2019-01-01 00:00:18,2703186189652095,37286541,28705
1,a1a22d70485983eac12b5b88dad1cf95,False,220.11,00:00:51,20190101,2019-01-01 00:00:51,38859492057661,16832522,83236
2,a41d7549acf90789359a9aa5346dcb46,False,41.96,00:03:06,20190101,2019-01-01 00:03:06,375534208663984,51244333,22844
3,189a841a0a8ba03058526bcfe566aab5,False,94.63,00:04:08,20190101,2019-01-01 00:04:08,4767265376804500,42189174,17972
4,6d294ed2cc447d2c71c7171a3d54967c,False,71.65,00:05:08,20190101,2019-01-01 00:05:08,6011360759745864,51228245,22644


In [48]:
category_dim = df_sample[['category_id','category']]
category_dim.head()

,category_id,category
0,0,misc_net
1,1,entertainment
2,2,misc_pos
3,3,gas_transport
4,3,gas_transport


In [49]:
merchant_dim = df_sample[['merch_zipcode','merchant','category_id']]
merchant_dim.head()

,merch_zipcode,merchant,category_id
0,28705,"fraud_Rippin, Kub and Mann",0
1,83236,fraud_Lind-Buckridge,1
2,22844,fraud_Keeling-Crist,2
3,17972,"fraud_Stroman, Hudson and Erdman",3
4,22644,fraud_Corwin-Collins,3


In [50]:
client_dim = df_sample[['cc_num','first','last','gender','job_id','age']]
client_dim.head()

,cc_num,first,last,gender,job_id,age
0,2703186189652095,Jennifer,Banks,F,1010,31
1,38859492057661,Edward,Sanchez,M,1011,57
2,375534208663984,Tyler,Garcia,M,1012,33
3,4767265376804500,Jennifer,Conner,F,1013,58
4,6011360759745864,Steven,Williams,M,1014,72


In [51]:
job_dim = df_sample[['job_id','job']]
job_dim.head()

,job_id,job
0,1010,"Psychologist, counselling"
1,1011,Nature conservation officer
2,1012,Dance movement psychotherapist
3,1013,Transport planner
4,1014,"Designer, multimedia"


In [52]:
date_dim = df_sample[['trans_date_id','date','month','year','quarter']]
date_dim.head()

,trans_date_id,date,month,year,quarter
0,20190101,2019-01-01,1,2019,1
1,20190101,2019-01-01,1,2019,1
2,20190101,2019-01-01,1,2019,1
3,20190101,2019-01-01,1,2019,1
4,20190101,2019-01-01,1,2019,1


In [53]:
location_dim = df_sample[['location_id','street','lat','long','zip','state_id']]
location_dim.head()

,location_id,street,lat,long,zip,state_id
0,37286541,561 Perry Cove,36.0788,-81.1781,28654,37
1,16832522,594 White Dale Suite 530,42.1808,-112.2620,83252,16
2,51244333,408 Bradley Rest,38.4207,-79.4629,24433,51
3,42189174,4655 David Island,40.3750,-75.2045,18917,42
4,51228245,231 Flores Pass Suite 720,38.8432,-78.6003,22824,51


In [54]:
state_dim = df_sample[['state_id','state_abbreviation','state_name','state_population']]
state_dim.head()

,state_id,state_abbreviation,state_name,state_population
0,37,NC,North Carolina,10439459
1,16,ID,Idaho,1839117
2,51,VA,Virginia,8631373
3,42,PA,Pennsylvania,13002788
4,51,VA,Virginia,8631373


# Saving the dimensions like CSV files

In [55]:
"""
This function removes duplicate data
from a specific DataFrame using the specified column.
"""
def dim_drop_duplicates(df, col):
    df.drop_duplicates(subset=[col], inplace=True)
    return df

In [56]:
"""
Removes duplicate data for each dimension before saving the changes
to CSV files.
"""

def save_dfs_to_csv(dfs, columns, filenames):
    """
    Apply dim_drop_duplicates on each DataFrame and save as a CSV file in a specific directory.

    Args:
        dfs (list): List of DataFrames to process.
        columns (list): List of column names for dropping duplicates in each DataFrame.
        filenames (list): List of filenames to save the CSVs.
    """
    # Folder where CSV files will be saved
    path_to_data_folder = os.path.join(os.getcwd(), '..', 'data', 'dimensions')

    # Ensure the folder exists, create it if it doesn't
    if not os.path.exists(path_to_data_folder):
        os.makedirs(path_to_data_folder)

    for df, col, filename in zip(dfs, columns, filenames):
        # Apply dim_drop_duplicates
        df = dim_drop_duplicates(df, col)
        
        # Create the full path for the CSV file
        full_path = os.path.join(path_to_data_folder, filename)
        
        # Save to CSV
        df.to_csv(full_path, index=False)
        print(f"Saved {filename} to {full_path}")

# List of DataFrames
dfs = [category_dim, merchant_dim, client_dim, job_dim, date_dim, location_dim, state_dim]

# Corresponding columns to check for duplicates
columns = ['category_id', 'merch_zipcode', 'cc_num', 'job_id', 'trans_date_id', 'location_id', 'state_id']

# Corresponding filenames
filenames = ['category_dim.csv', 'merchant_dim.csv', 'client_dim.csv', 'job_dim.csv', 'date_dim.csv', 'location_dim.csv', 'state_dim.csv']

# Call the function to process and save each DataFrame
save_dfs_to_csv(dfs, columns, filenames)


C:\Users\david\AppData\Local\Temp\ipykernel_25068\3067145849.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=[col], inplace=True)


Saved category_dim.csv to c:\Users\david\Desktop\credit-Card-Transaction-ETL-Project\notebooks\..\data\dimensions\category_dim.csv
Saved merchant_dim.csv to c:\Users\david\Desktop\credit-Card-Transaction-ETL-Project\notebooks\..\data\dimensions\merchant_dim.csv
Saved client_dim.csv to c:\Users\david\Desktop\credit-Card-Transaction-ETL-Project\notebooks\..\data\dimensions\client_dim.csv
Saved job_dim.csv to c:\Users\david\Desktop\credit-Card-Transaction-ETL-Project\notebooks\..\data\dimensions\job_dim.csv
Saved date_dim.csv to c:\Users\david\Desktop\credit-Card-Transaction-ETL-Project\notebooks\..\data\dimensions\date_dim.csv
Saved location_dim.csv to c:\Users\david\Desktop\credit-Card-Transaction-ETL-Project\notebooks\..\data\dimensions\location_dim.csv
Saved state_dim.csv to c:\Users\david\Desktop\credit-Card-Transaction-ETL-Project\notebooks\..\data\dimensions\state_dim.csv


In [63]:
# save fact table
print(fact_T_transation_dim.shape)
fact_T_transation_dim_copy = fact_T_transation_dim.head(800000)
print(fact_T_transation_dim_copy.shape)
# guardar fact table
fact_T_transation_dim_copy.to_csv(os.path.join(os.getcwd(), '..', 'data', 'dimensions', 'fact_T_transation_dim.csv'), index=False)

(1052352, 9)
(800000, 9)


In [13]:
import pandas as pd
import os

# Load the data into a DataFrame

df = pd.read_csv("../data/sample_credit_card_transactions_api_preprocessed.csv")

In [11]:
# Eliminar 
df.shape

(300000, 26)

In [12]:
# save df to csv
df.to_csv(os.path.join(os.getcwd(), '..', 'data', 'df.csv'), index=False)